In [12]:
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')
le = LabelEncoder()
from sklearn.metrics import classification_report

In [2]:
import gzip
import shutil

input_path = '/content/drive/MyDrive/ML/INTERMEDIATE/ML_INT_Q10/train.gz'
output_path = '/content/AD_AGENCY.csv'

with gzip.open(input_path, 'rb') as f_in:
    with open(output_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)


In [5]:
file = open("/content/AD_AGENCY.csv")
n = sum(1 for line in file) - 1
s = 100000
skip = sorted(random.sample(range(1, n+1), n-s))
chunk = pd.read_csv("/content/AD_AGENCY.csv", low_memory=False, chunksize=10000, skiprows=skip )
df = pd.concat(chunk)



In [6]:
df

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,10118071069040875175,0,14102100,1005,0,b87f11a1,0b204815,3e814130,ecad2386,7801e8d9,...,1,0,16920,320,50,1899,0,431,-1,117
1,10157267783354901009,0,14102100,1005,1,43d6df75,27e3c518,28905ebd,ecad2386,7801e8d9,...,1,0,15701,320,50,1722,0,35,-1,79
2,10160139270245386403,0,14102100,1005,0,5b08c53b,7687a86e,3e814130,ecad2386,7801e8d9,...,1,0,19016,300,250,2162,2,39,100248,33
3,10172203585246420371,0,14102100,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,...,1,0,20366,320,50,2333,0,39,-1,157
4,10173866029588856830,0,14102100,1005,0,02d5151c,64778742,f028772b,ecad2386,7801e8d9,...,1,0,20596,320,50,2161,0,35,100105,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,947060696585742433,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,5e657d7f,2347f47a,...,1,2,23018,320,50,2658,3,35,100148,23
99996,9713884098303501568,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,e2fcccd2,5c5a694b,...,1,0,6559,320,50,571,2,39,100049,32
99997,991100654637317914,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,3c4b944d,2347f47a,...,1,0,18987,320,50,2158,3,291,100189,61
99998,9914133453817176506,0,14103023,1005,0,2f19f7f2,7cfb8bd7,3e814130,ecad2386,7801e8d9,...,1,0,23925,320,50,2742,0,163,-1,17


In [7]:
# Dealing with outliers by capping

col = ['C15', 'C16', 'C19', 'C21']
for col in col:
    percentiles = df[col].quantile(0.98)
    if df[col].quantile(0.98) < 0.5 * df[col].max():
        df[col][df[col] >= percentiles] = percentiles
numerical = []
categorical = []

for col in df.columns:
    if df[col].dtype == "object":
        categorical.append(col)
    else:
        numerical.append(col)

In [8]:
for i in categorical:
    df[i] = le.fit_transform(df[i])
print(df.click.value_counts(normalize=True))
print("\n")
df.drop(['C20'], axis=1, inplace=True)
df.drop(['id', 'hour'], axis=1, inplace = True)
df.rename(columns={'click': 'y'}, inplace=True, errors='raise')

0    0.83082
1    0.16918
Name: click, dtype: float64




In [9]:
def model_eval(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
    return accuracy, cnf_matrix

In [13]:
X = df.drop(['y'], axis=1)
y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size= 0.3, random_state= 42)
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X_train, y_train)
y_preds = model_rf.predict(X_test)
accuracy, cnf_matrix = model_eval(model_rf, X_test, y_test)
print(f"accuracy : {accuracy}")
print(f"cnf_matrix : {cnf_matrix}")

accuracy : 0.8079333333333333
cnf_matrix : [[23434  1491]
 [ 4271   804]]


In [14]:

print(classification_report(y_preds, y_test))

              precision    recall  f1-score   support

           0       0.94      0.85      0.89     27705
           1       0.16      0.35      0.22      2295

    accuracy                           0.81     30000
   macro avg       0.55      0.60      0.55     30000
weighted avg       0.88      0.81      0.84     30000

